# Prototype of lipreading pipeline

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

import random
import math
import time

# model file, encoder, decoder and seqtoseq
from model import *
# utils file
from utils import *
# Get landmark using vocadataset.py
from data.vocaset import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# Get landmark from vocadaset class
#trainset = vocadataset("train", landmark=True)
trainset = vocadataset("train", landmark=True, mouthOnly=True)
landmark, labels = trainset[0]

In [3]:
vocabulary = create_vocabulary(blank="@")

In [4]:
# Create a mapping from characters to indices
char_to_index = {char: index for index, char in enumerate(vocabulary)}
print(char_to_index)

{'@': 0, '#': 1, 'a': 2, 'b': 3, 'c': 4, 'd': 5, 'e': 6, 'f': 7, 'g': 8, 'h': 9, 'i': 10, 'j': 11, 'k': 12, 'l': 13, 'm': 14, 'n': 15, 'o': 16, 'p': 17, 'q': 18, 'r': 19, 's': 20, 't': 21, 'u': 22, 'v': 23, 'w': 24, 'x': 25, 'y': 26, 'z': 27, ' ': 28}


In [5]:
# Convert the sequence and target to indices
#sequence_indices = [char_to_index[char] for char in sequence]
target_indices = [char_to_index[char] for char in labels]
target_tensor = torch.tensor(target_indices)

In [6]:
INPUT_DIM = 36*3
INPUT_DIM = landmark.shape[1]*3
HID_DIM = 32
output_dim = len(vocabulary)

enc = Encoder(INPUT_DIM, HID_DIM, num_layers=1)
dec = Decoder(output_dim, HID_DIM, num_layers=1)
model = Seq2Seq(enc, dec, 'cpu')#.to(device)

reshaped_landmark = torch.reshape(landmark, (landmark.shape[0], landmark.shape[1]*landmark.shape[2]))


In [9]:
# Define the CTC loss function
ctc_loss = nn.CTCLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = model(reshaped_landmark[None, : , :], target_tensor[None, None,:])
    

    input_lengths = torch.full((1,), output.size(0), dtype=torch.long)
    target_lengths = torch.full((target_tensor.size(0),), target_tensor.size(0), dtype=torch.long)
    
    loss = ctc_loss(torch.nn.functional.log_softmax(output, dim=2), target_tensor, input_lengths, target_lengths[0])
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")
        e = torch.argmax(output, dim=2).squeeze(1)
        output_sequence = ''.join([vocabulary[index] for index in e])
        print(output_sequence)



Epoch [10/1000], Loss: 1.7837450504302979
she haddd youurrrrrr    @s@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Epoch [20/1000], Loss: 2.025465726852417
she hadd yourrrrr                         @s@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@


KeyboardInterrupt: 

In [32]:
print(output[0].max())
#print(torch.argmax(output, dim=2).squeeze(1)[0])
output.shape
torch.argmax(output, dim=2).squeeze(1)


tensor(-0.3272, grad_fn=<MaxBackward1>)


tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0])

In [ ]:
# Decode the output sequence
output_indices = torch.argmax(output, dim=2).squeeze(1)
output_sequence = ''.join([vocabulary[index] for index in output_indices])

print("Target Sequence:", labels.replace("@","").replace("#",""))
print("Decoded Output:", process_string(output_sequence))